<a href="https://colab.research.google.com/github/manan180796/Citation/blob/main/GAT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install spektral

     |████████████████████████████████| 102kB 6.8MB/s 


This example implements the experiments on citation networks from the paper:
Graph Attention Networks (https://arxiv.org/abs/1710.10903)
Petar Veličković, Guillem Cucurull, Arantxa Casanova, Adriana Romero, Pietro Liò, Yoshua Bengio

In [3]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

from spektral.datasets import citation
from spektral.layers import GraphAttention

In [4]:

# Load data
dataset = 'cora'
A, X, y, train_mask, val_mask, test_mask = citation.load_data(dataset)

# Parameters
channels = 8            # Number of channel in each head of the first GAT layer
n_attn_heads = 8        # Number of attention heads in first GAT layer
N = X.shape[0]          # Number of nodes in the graph
F = X.shape[1]          # Original size of node features
n_classes = y.shape[1]  # Number of classes
dropout = 0.6           # Dropout rate for the features and adjacency matrix
l2_reg = 5e-6           # L2 regularization rate
learning_rate = 5e-3    # Learning rate
epochs = 20000          # Number of training epochs
es_patience = 100       # Patience for early stopping

Loading cora dataset
Pre-processing node features


In [5]:
# Model definition
X_in = Input(shape=(F, ))
A_in = Input(shape=(N, ), sparse=True)

dropout_1 = Dropout(dropout)(X_in)
graph_attention_1 = GraphAttention(channels,
                                   attn_heads=n_attn_heads,
                                   concat_heads=True,
                                   dropout_rate=dropout,
                                   activation='elu',
                                   kernel_regularizer=l2(l2_reg),
                                   attn_kernel_regularizer=l2(l2_reg)
                                   )([dropout_1, A_in])
dropout_2 = Dropout(dropout)(graph_attention_1)
graph_attention_2 = GraphAttention(n_classes,
                                   attn_heads=1,
                                   concat_heads=False,
                                   dropout_rate=dropout,
                                   activation='softmax',
                                   kernel_regularizer=l2(l2_reg),
                                   attn_kernel_regularizer=l2(l2_reg)
                                   )([dropout_2, A_in])
# Build model
model = Model(inputs=[X_in, A_in], outputs=graph_attention_2)

In [6]:
optimizer = Adam(lr=learning_rate)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              weighted_metrics=['acc'])
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1433)]       0                                            
__________________________________________________________________________________________________
dropout (Dropout)               (None, 1433)         0           input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 2708)]       0                                            
__________________________________________________________________________________________________
graph_attention (GraphAttention (None, 64)           91904       dropout[0][0]                    
                                                                 input_2[0][0]         

In [8]:
# Preprocessing operations
A = A.astype('f4')
X = X.toarray()
validation_data = ([X, A], y, val_mask)

In [9]:

model.fit([X, A],
          y,
          sample_weight=train_mask,
          epochs=epochs,
          batch_size=N,
          validation_data=validation_data,
          shuffle=False,  # Shuffling data means shuffling the whole graph
          callbacks=[
              EarlyStopping(patience=es_patience, restore_best_weights=True)
          ])

Epoch 1/20000


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


1/1 [==============================] - 1s 947ms/step - loss: 0.1007 - acc: 0.1643 - val_loss: 0.3595 - val_acc: 0.2040
Epoch 2/20000
1/1 [==============================] - 0s 66ms/step - loss: 0.1006 - acc: 0.2071 - val_loss: 0.3593 - val_acc: 0.2200
Epoch 3/20000
1/1 [==============================] - 0s 66ms/step - loss: 0.1005 - acc: 0.3000 - val_loss: 0.3590 - val_acc: 0.2380
Epoch 4/20000
1/1 [==============================] - 0s 66ms/step - loss: 0.1005 - acc: 0.3286 - val_loss: 0.3584 - val_acc: 0.2040
Epoch 5/20000
1/1 [==============================] - 0s 68ms/step - loss: 0.1003 - acc: 0.3571 - val_loss: 0.3580 - val_acc: 0.3860
Epoch 6/20000
1/1 [==============================] - 0s 66ms/step - loss: 0.1002 - acc: 0.3214 - val_loss: 0.3576 - val_acc: 0.5880
Epoch 7/20000
1/1 [==============================] - 0s 67ms/step - loss: 0.1003 - acc: 0.2857 - val_loss: 0.3573 - val_acc: 0.6340
Epoch 8/20000
1/1 [==============================] - 0s 66ms/step - loss: 0.0999 - acc: 0

In [10]:
# Evaluate model
print('Evaluating model.')
eval_results = model.evaluate([X, A],
                              y,
                              sample_weight=test_mask,
                              batch_size=N)
print('Done.\n'
      'Test loss: {}\n'
      'Test accuracy: {}'.format(*eval_results))

Evaluating model.
1/1 [==============================] - 0s 2ms/step - loss: 0.2290 - acc: 0.8350
Done.
Test loss: 0.22903212904930115
Test accuracy: 0.8349999785423279
